In [7]:
import pandas as pd
import numpy as np
import os

In [8]:

file_path = r"C:\Users\surji\Desktop\Quant_Poject\Perfomance_metrics\data\pe_ratio_rolling_analysis.csv"
df = pd.read_csv(file_path, parse_dates=['Date'])
initial_capital = 100_000
tickers = df['TickerName'].unique()
rolling_windows = df['RollingWindow'].unique()



Calculation of MDD and Sharpe Ratio 

In [ ]:

def calculate_max_drawdown(cum_returns):
    running_max = np.maximum.accumulate(cum_returns)
    drawdown = (cum_returns - running_max) / running_max
    max_drawdown = drawdown.min()
    return max_drawdown

def calculate_sharpe_ratio(returns, risk_free_rate=0, periods_per_year=252):
    excess_returns = returns - risk_free_rate/periods_per_year
    if returns.std() == 0:
        return 0
    sharpe = np.sqrt(periods_per_year) * excess_returns.mean() / returns.std()
    return sharpe


In [10]:

results = []

for window in rolling_windows:
    for ticker in tickers:
        temp = df[(df['TickerName'] == ticker) & (df['RollingWindow'] == window)].copy()
        temp = temp.sort_values('Date').reset_index(drop=True)

        temp['Return'] = temp['StockPrice'].pct_change().fillna(0)
        temp['Position'] = 0
        holding = False

        for i in range(1, len(temp)):
            if temp.loc[i-1, 'Signal'] == 1:
                holding = True
            elif temp.loc[i-1, 'Signal'] == -1:
                holding = False
            temp.loc[i, 'Position'] = 1 if holding else 0

        temp['Strategy_Return'] = temp['Return'] * temp['Position']
        temp['Cumulative_Market'] = (1 + temp['Return']).cumprod()
        temp['Cumulative_Strategy'] = (1 + temp['Strategy_Return']).cumprod()


        roi = temp['Cumulative_Strategy'].iloc[-1] - 1  # Total return
        max_dd = calculate_max_drawdown(temp['Cumulative_Strategy'].values)
        sharpe = calculate_sharpe_ratio(temp['Strategy_Return'].values)
        roi_to_mdd = roi / abs(max_dd) if max_dd != 0 else np.nan

        results.append({
            'TickerName': ticker,
            'RollingWindow': window,
            'ROI': roi,
            'MaxDrawdown': max_dd,
            'SharpeRatio': sharpe,
            'ROI_to_MDD': roi_to_mdd
        })


In [15]:
results_df = pd.DataFrame(results)

folder_path = r"C:\Users\surji\Desktop\Quant_Poject\Perfomance_metrics\data"
file_name = "experiment_tracking.csv"
full_path = os.path.join(folder_path, file_name)

results_df.to_csv(full_path, index=False)
print(f"Backtesting complete. Results are also saved in the experiment tarcking file\n")
print(results_df.head())


Backtesting complete. Results are also saved in the experiment tarcking file

  TickerName  RollingWindow       ROI  MaxDrawdown  SharpeRatio  ROI_to_MDD
0       AAPL             20  2.439628    -0.395730     3.887596    6.164883
1      GOOGL             20  7.225410    -0.390930     5.903248   18.482623
2       META             20  3.763979    -0.653914     4.138042    5.756077
3       MSFT             20  1.234127    -0.070856     4.086974   17.417498
4       TSLA             20  0.000000     0.000000     0.000000         NaN


This is just for reference to see what signals are getting generated and from what date 

In [12]:
print(temp[['Date', 'Signal']].tail(30))  # Check recent signal values


         Date  Signal
29 2018-06-30       0
30 2018-09-30       0
31 2018-12-31       0
32 2019-03-31       0
33 2019-06-30       0
34 2019-09-30       0
35 2019-12-31       0
36 2020-03-31       0
37 2020-06-30      -1
38 2020-09-30      -1
39 2020-12-31      -1
40 2021-03-31      -1
41 2021-06-30      -1
42 2021-09-30       0
43 2021-12-31       0
44 2022-03-31       0
45 2022-06-30       0
46 2022-09-30       0
47 2022-12-31       0
48 2023-03-31       0
49 2023-06-30       0
50 2023-09-30       0
51 2023-12-31       0
52 2024-03-31       0
53 2024-06-30       0
54 2024-09-30       0
55 2024-12-31       0
56 2025-03-31       0
57 2025-06-30       0
58 2025-07-25       0


This is classfication of the buy signals and sell signals 

In [13]:
entry_dates = temp[temp['Signal'] == 1]['Date']
exit_dates = temp[temp['Signal'] == -1]['Date']
print("Buy signals:\n", entry_dates)
print("Sell signals:\n", exit_dates)


Buy signals:
 Series([], Name: Date, dtype: datetime64[ns])
Sell signals:
 37   2020-06-30
38   2020-09-30
39   2020-12-31
40   2021-03-31
41   2021-06-30
Name: Date, dtype: datetime64[ns]
